In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import euclidean_distances
from collections import Counter

In [ ]:
# Load MNIST dataset and concatenate train and test sets
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Concatenation of split data
X = np.concatenate((X_train, X_test), axis=0)  # X contains all images
y = np.concatenate((y_train, y_test), axis=0)  # y contains all labels

# Flatten the images (from 28x28 to 784)
data = X.reshape(X.shape[0], -1)

# Normalize the dataset (scaling pixel values to [0,1])
data = data / 255.0

In [ ]:
# Function to initialize centroids randomly from the dataset
def init_centroids(data, k):
    np.random.seed(42)  # Set seed for reproducibility
    return data[np.random.choice(data.shape[0], k, replace=False)]  # Select k random points

In [ ]:
# Soft K-Means E-step: Compute soft memberships using distances and beta parameter
# In contraste with the hard KMeans, in this step I compute the prob of a point to belong to a cluster
# using a Pr function (exponential function) controlled by the beta parameter

def e_step_soft(data, centroids, beta):
    # compute Euclidean distances between each data point and each centroid
    distances = euclidean_distances(data, centroids)

    # square the distances (since similarity decreases with distance squared)
    squared_distances = distances ** 2

    # compute negative beta-scaled distances (logits for softmax)
    #transform distances in negative values in order to make the smallest one hihgers helping to compute similarities
    logits = -beta * squared_distances

    # normalize by subtracting the max value per row (to deal w/ big values after exp)
    max_logits = np.max(logits, axis=1, keepdims=True)

    # Compute exponentials of the scaled distances
    exp_logits = np.exp(logits - max_logits)

    # Normalize to get probabilities (soft cluster assignments)
    soft_memberships = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

    return soft_memberships


In [ ]:
# Compute the objective function J for soft K-Means
def compute_objective_soft(data, centroids, memberships):
    dist_squared = euclidean_distances(data, centroids) ** 2
    return np.sum(memberships * dist_squared)  # Weighted sum of squared distances

In [ ]:
# Soft M-step: Update centroids using soft memberships
def m_step_soft(data, memberships):
    return np.dot(memberships.T, data) / np.sum(memberships, axis=0, keepdims=True).T  # Weighted mean update


In [ ]:
# soft K-Means main function
def soft_kmeans(data, k, beta, max_iter=10):
    centroids = init_centroids(data, k)  # Initialize centroids
    J_history = []  # Store objective function values

    for iteration in range(max_iter):
        # E-step: compute soft memberships
        memberships = e_step_soft(data, centroids, beta)
        J_e = compute_objective_soft(data, centroids, memberships)
        J_history.append((iteration + 1, 'E-step', J_e))
        print(f"Iteration {iteration + 1} (E-step), Objective Function J: {J_e}")

        # M-step: update centroids
        centroids = m_step_soft(data, memberships)
        J_after = compute_objective_soft(data, centroids, memberships)
        J_history.append((iteration + 1, 'M-step', J_after))
        print(f"Iteration {iteration + 1} (M-step), Objective Function J: {J_after}")

    return centroids, memberships, J_history

In [ ]:
# Run Soft K-Means with different beta values
k = 10  # Number of clusters
max_iter = 10  # Number of iterations

for beta in [0.1, 1, 10]:  # Test different values of beta
#low beta: more difuse assigment. Low Purity, high Gini
#high beta: more hard clustering. High Purity, Low Gini
    print(f"\nRunning Soft K-Means with beta={beta}")
    centroids, memberships, J_values = soft_kmeans(data, k, beta, max_iter)

    # Retrieve soft cluster assignments (most probable cluster for each point)
    clusters = np.argmax(memberships, axis=1)

    # Compute purity
    def purity(clusters, labels):
        correct_labels = 0
        n_clusters = len(np.unique(clusters))
        for cluster in range(n_clusters):
            indices = np.where(clusters == cluster)[0]
            cluster_labels = labels[indices]
            most_freq = Counter(cluster_labels).most_common(1)[0][1] if len(cluster_labels) > 0 else 0
            correct_labels += most_freq
        return correct_labels / len(labels)

    mnist_purity = purity(clusters, y)
    print(f"Purity for beta={beta}: {mnist_purity}")

    # Compute Gini index
    def gini_index(clusters, labels):
        n_samples = len(labels)
        n_clusters = len(np.unique(clusters))
        total_gini = 0
        for cluster in range(n_clusters):
            indices = np.where(clusters == cluster)[0]
            cluster_labels = labels[indices]
            total_points = len(cluster_labels)
            if total_points == 0:
                continue
            label_counts = Counter(cluster_labels)
            proportions = np.array([count / total_points for count in label_counts.values()])
            gini = 1 - np.sum(proportions**2)
            total_gini += total_points * gini
        return total_gini / n_samples

    gini_mnist = gini_index(clusters, y)
    print(f"Gini Index for beta={beta}: {gini_mnist}")


Running Soft K-Means with beta=0.1
Iteration 1 (E-step), Objective Function J: 5550719.149770475
Iteration 1 (M-step), Objective Function J: 3325676.4676735066
Iteration 2 (E-step), Objective Function J: 3515893.0570897684
Iteration 2 (M-step), Objective Function J: 3457536.6174354353
Iteration 3 (E-step), Objective Function J: 3515197.661252457
Iteration 3 (M-step), Objective Function J: 3503089.659505628
Iteration 4 (E-step), Objective Function J: 3530706.840236037
Iteration 4 (M-step), Objective Function J: 3526140.6917369524
Iteration 5 (E-step), Objective Function J: 3541755.3993361616
Iteration 5 (M-step), Objective Function J: 3539540.9462528806
Iteration 6 (E-step), Objective Function J: 3549131.2696833275
Iteration 6 (M-step), Objective Function J: 3547915.276606634
Iteration 7 (E-step), Objective Function J: 3554106.412412989
Iteration 7 (M-step), Objective Function J: 3553382.620266587
Iteration 8 (E-step), Objective Function J: 3557498.6452415343
Iteration 8 (M-step), Obje